In [14]:
import csv
import copy

In [15]:
DCT_8pt_lib = csv.reader(open("DCT_8pt_lib.txt", 'r'))
DCT_8pt = "DCT_8pt.txt"
file1 = DCT_8pt_lib
file_ = DCT_8pt

In [16]:
file = []
for i in file1:
    file.append(i)

In [17]:
def application(file):
    minadder, minmulti = 1,1                                                         #r1min r2min  
    maxadder, maxmulti = 0,0                                                         #initialized with 0
    arAdder,arMulti, arRegister, delayAdder, delayMulti = 0,0,0,0,0                  #initialized with 0
    for i in file:
        if len(i)!=0:
            if i[0] == 'adder(16)' and maxadder<int(i[-1]):
                maxadder = int(i[-1])
                arAdder = eval(i[2][:-2])
                delayAdder = eval(i[3][:-2])
            elif i[0] == 'multiplier(16)' and maxmulti<int(i[-1]):
                maxmulti = int(i[-1])
                arMulti = eval(i[2][:-2])
                delayMulti = eval(i[3][:-2])
            elif i[0].find("register") != -1:
                arRegister = eval(i[0].split()[1][:-2])
    #print(arAdder,arMulti, arRegister, delayAdder, delayMulti)
    return (minadder, minmulti, maxadder, maxmulti, arAdder,arMulti, arRegister, delayAdder, delayMulti)

In [18]:
minadder, minmulti, maxadder, maxmulti, arAdder,arMulti, arRegister, delayAdder, delayMulti = application(file)

In [19]:
class Node:
    def __init__(self, s):
        self.operator = s[0]
        self.operand_1 = int(s[1])
        self.operand_2 = int(s[2])
        self.id = int(s[3])
        self.control_step = None

In [20]:
def read_file(s):
    file = open(s)
    fir = file.read()
    file.close()
    fir_dfg = ""
    for i in fir:
        if i != '\n':
            fir_dfg += i
    fir_dfg = fir_dfg.split(",")
    return fir_dfg

In [21]:
def resource_constraint(sign, resources):
    list_of_nodes = []
    for i in range(int(len(sign)/4)):
        list_of_nodes.append(Node(sign[i*4:i*4+4]))
    operators = {}
    for i in list_of_nodes:
        operators[i.operator] = None
    for i in operators:
        if i == '+':
            operators[i] = resources[0]
        if i == '*':
            operators[i] = resources[1]
    return operators, list_of_nodes

In [22]:
def scheduling(operators, list_of_nodes):
    c_oper = {}
    t_op = {}
    count = len(list_of_nodes)
    list_of = copy.deepcopy(list_of_nodes)
    for i in range(count):
        t_oper = {}
        for j in operators:
            t_oper[j] = operators[j]
        t_op[i] = t_oper
    for i in operators:
        c_oper[i] = 1
    
    t_oper = {}
    for i in operators:
        t_oper[i] = operators[i]

    for node in list_of_nodes:
        if node.operand_1 == 0 and node.operand_2 == 0:
            if t_oper[node.operator] == 0:
                c_oper[node.operator] += 1
                t_oper[node.operator] = operators[node.operator]
            node.control_step = c_oper[node.operator]
            t_oper[node.operator] -= 1
            t_op[node.control_step-1][node.operator] -= 1
            count -= 1
    
    while count != 0:
        for node in list_of_nodes:
            if node.control_step == None:
                if node.operand_1 == 0 and node.operand_2 != 0:
                    if list_of_nodes[node.operand_2 - 1].control_step != None:
                        max_c = max(0, list_of_nodes[node.operand_2 - 1].control_step)
                        while((t_op[max_c][node.operator]) == 0) :
                            max_c += 1
                        if (((1 + max_c) > c_oper[node.operator]) or (t_op[max_c][node.operator]) != 0):
                                c_oper[node.operator] = 1 + max_c
                                t_oper[node.operator] = t_op[max_c][node.operator]
                                node.control_step = c_oper[node.operator]
                        node.control_step = c_oper[node.operator]
                        t_oper[node.operator] -= 1
                        t_op[node.control_step-1][node.operator] -= 1
                        count -= 1
                        
                elif node.operand_1 != 0 and node.operand_2 == 0:
                    if list_of_nodes[node.operand_1 - 1].control_step != None:
                        max_c = max(0, list_of_nodes[node.operand_1 - 1].control_step)
                        while((t_op[max_c][node.operator]) == 0) :
                            max_c += 1
                        if (((1 + max_c) > c_oper[node.operator]) or (t_op[max_c][node.operator]) != 0):
                                c_oper[node.operator] = 1 + max_c
                                t_oper[node.operator] = t_op[max_c][node.operator]
                                node.control_step = c_oper[node.operator]
                        node.control_step = c_oper[node.operator]
                        t_oper[node.operator] -= 1
                        t_op[node.control_step-1][node.operator] -= 1
                        count -= 1
                       
                else:
                    if list_of_nodes[node.operand_1 - 1].control_step != None and list_of_nodes[node.operand_2 - 1].control_step != None:
                        max_c = max(list_of_nodes[node.operand_1 - 1].control_step, list_of_nodes[node.operand_2 - 1].control_step)
                        while((t_op[max_c][node.operator]) == 0) :
                            max_c += 1
                        if (((1 + max_c) > c_oper[node.operator]) or (t_op[max_c][node.operator]) != 0):
                                c_oper[node.operator] = 1 + max_c
                                t_oper[node.operator] = t_op[max_c][node.operator]
                                node.control_step = c_oper[node.operator]
                        node.control_step = c_oper[node.operator]
                        t_oper[node.operator] -= 1
                        t_op[node.control_step-1][node.operator] -= 1
                        count -= 1
                       
    max_con = 0
    for i in list_of_nodes:
        if i.control_step > max_con:
            max_con = i.control_step
    return list_of, list_of_nodes, max_con

In [23]:
s_i = read_file(file_)
operators, list_of_nodes = resource_constraint(s_i, [3,3])
list_of, list_of_nodes, max_con = scheduling(operators, list_of_nodes)

In [24]:
for i in list_of_nodes:
    print("Operator, Id: ",i.id,i.operator, "Control step: ",i.control_step)

Operator, Id:  1 * Control step:  1
Operator, Id:  2 * Control step:  1
Operator, Id:  3 * Control step:  1
Operator, Id:  4 * Control step:  2
Operator, Id:  5 * Control step:  2
Operator, Id:  6 * Control step:  2
Operator, Id:  7 * Control step:  3
Operator, Id:  8 * Control step:  3
Operator, Id:  9 + Control step:  2
Operator, Id:  10 + Control step:  3
Operator, Id:  11 + Control step:  4
Operator, Id:  12 + Control step:  5
Operator, Id:  13 + Control step:  6
Operator, Id:  14 + Control step:  7
Operator, Id:  15 + Control step:  8
